In [62]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from typing import Dict, Tuple
import re
import keras.layers as l
from keras import models, callbacks, utils, losses

In [66]:
text = ''
with open('Пикник на обочине.txt', 'r', encoding='windows-1251') as file:
    text = file.read()

def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

words = list(filter(None, [re.sub('[^а-яА-ЯёЁ0-9 ,-]', '', s).strip() for s in text.split('.')]))
alphabet = np.array(sorted(set(' '.join(words).split(' '))))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[word] for word in ' '.join(words).split()])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)


In [72]:
model = keras.Sequential([
    l.Embedding(len(alphabet), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.SimpleRNN(128, return_sequences=True, stateful=True),
    l.Dense(len(alphabet) / 2, activation='relu'),
    l.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=50, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/50
44/44 [==============================] - 15s 308ms/step - loss: 8.7083 - accuracy: 0.0278
Epoch 2/50
44/44 [==============================] - 12s 264ms/step - loss: 7.9602 - accuracy: 0.0341
Epoch 3/50
44/44 [==============================] - 12s 276ms/step - loss: 8.0234 - accuracy: 0.0326
Epoch 4/50
44/44 [==============================] - 12s 266ms/step - loss: 7.8736 - accuracy: 0.0364
Epoch 5/50
44/44 [==============================] - 11s 260ms/step - loss: 7.9238 - accuracy: 0.0359
Epoch 6/50
44/44 [==============================] - 11s 261ms/step - loss: 7.5718 - accuracy: 0.0369
Epoch 7/50
44/44 [==============================] - 11s 261ms/step - loss: 6.9470 - accuracy: 0.0414
Epoch 8/50
44/44 [==============================] - 11s 260ms/step - loss: 5.9504 - accuracy: 0.0507
Epoch 9/50
44/44 [==============================] - 12s 263ms/step - loss: 4.6936 - accuracy: 0.2137
Epoch 10/50
44/44 [==============================] - 12s 265ms/step - loss: 3.9782 - accura

In [73]:
def predict_next(sample: str, model: keras.Sequential, tokenizer: Dict[str, int], vocabulary: Dict[int, str], n_next: int, temperature: float, batch_size: int, word: bool = False) -> str:
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [79]:
print(predict_next(
    sample='Счастье',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

Счастье были говорю его какой-то себя по институт его негромко сказал Рэдрик Не несут такой Он Гуталин еще нет, еще не


In [80]:
print(predict_next(
    sample='Разум',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

Разум у него то что был это не будем не ходил Я свое говорю, уже не было, а ты затем слова


In [85]:
print(predict_next(
    sample='Шухарт',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

Шухарт А парень, что это за новости И бы в жизни что, только как пытается Да и Вот через сел за
